In [1]:
# Testing ml_processor

In [2]:
from tensorflow import keras

In [3]:
# This function will load a model based on the path and model_name provided
def load_model(model_name: str, path="./"):
    try:
        model_path = f"{path}/{model_name}.keras"
        loaded_model = keras.models.load_model(model_path)
        return loaded_model
    except Exception as e:
        print(e)

In [4]:
loaded_model = load_model('dmd_lstm')

In [5]:
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 6         
                                                                 
Total params: 6
Trainable params: 6
Non-trainable params: 0
_________________________________________________________________


In [6]:
import numpy as np

In [7]:
def process_data(loaded_model, model_name:str, len_predictions:int=5):
    try:
        # Iterate through the 20 stock data files from /data/db/stock_data/{stock_symbol}.csv
        stock_symbols = stock_symbols = [
    'MEG',
    'JGS',
    'BDO',
    'FGEN',
    'ICT',
    'ALI',
    'SMC',
    'TEL',
    'GLO',
    'BLOOM',
    'RLC',
    'MER',
    'AC',
    'PGOLD',
    'LTG',
    'MPI',
    'AP',
    'RRHI',
    'URC'
]
        # Appeend PSEI to the list of stock symbols
        stock_symbols.append("PSEI")

        # Initialize the dictionary that will hold the processed data
        # key = stock symbol
        # value = (last actual closing, list of predictions with length=predict_len)
        processed_data = {stock:[] for stock in stock_symbols}

        # Iterate through the stock symbols
        for stock in stock_symbols:
            predicted_closings = []
            
            # Initialize the closing prices of the stock
            closing_prices = np.array([])
            # Get the last closing prices based on the window size of the model
            window_size = loaded_model.count_params() - 1
            closing_prices = np.genfromtxt(f'./data_test/{stock}.csv',
                                            delimiter=',',
                                            skip_header=1,
                                            usecols=4)
            closing_prices = closing_prices[-(window_size):]
            # Get the last actual closing price
            last_actual = closing_prices[-1]

            """
                PREDICTED CLOSINGS:
                    1st Prediction: input[closing_prices]
                    2nd Prediction: input[closing_prices[-4:], pred]
                    3rd Prediction: input[closing_prices[-3:], pred]
                    4th Prediction: input[closing_prices[-2:], pred]
                    5th Prediction: input[closing_prices[-1:], pred]
            """
            for _ in range(len_predictions):
                data = np.append(closing_prices, predicted_closings)
                prediction = loaded_model.predict(data[-(window_size):].reshape(1,5))
                predicted_closings.append(float(prediction))

            processed_data[stock] = tuple((last_actual, {model_name: predicted_closings}))

        return processed_data
    
    except Exception as e:
        print(e)

In [8]:
processed_data = process_data(loaded_model=loaded_model, model_name="dmd_lstm")

1/1 [==============================] - 0s 22ms/step


In [9]:
processed_data

{'MEG': (2.34,
  {'dmd_lstm': [2.3027100563049316,
    2.372716188430786,
    2.3127152919769287,
    2.342719316482544,
    2.3054213523864746]}),
 'JGS': (53.9,
  {'dmd_lstm': [53.902706146240234,
    54.002685546875,
    53.602657318115234,
    53.90264892578125,
    53.905338287353516]}),
 'BDO': (125.5,
  {'dmd_lstm': [124.00247955322266,
    126.00253295898438,
    125.00251007080078,
    125.50272369384766,
    124.00498962402344]}),
 'FGEN': (18.02,
  {'dmd_lstm': [18.382694244384766,
    18.10270118713379,
    18.182720184326172,
    18.022674560546875,
    18.385387420654297]}),
 'ICT': (214.0,
  {'dmd_lstm': [206.6023406982422,
    212.00233459472656,
    211.40199279785156,
    214.00294494628906,
    206.60494995117188]}),
 'ALI': (29.25,
  {'dmd_lstm': [29.052684783935547,
    29.202686309814453,
    29.102670669555664,
    29.2526912689209,
    29.05536651611328]}),
 'SMC': (107.5,
  {'dmd_lstm': [114.60272216796875,
    112.70266723632812,
    110.00323486328125,
    10

In [10]:
last_actual_closing = {stock:processed_data[stock][1] for stock in processed_data}
last_actual_closing

{'MEG': {'dmd_lstm': [2.3027100563049316,
   2.372716188430786,
   2.3127152919769287,
   2.342719316482544,
   2.3054213523864746]},
 'JGS': {'dmd_lstm': [53.902706146240234,
   54.002685546875,
   53.602657318115234,
   53.90264892578125,
   53.905338287353516]},
 'BDO': {'dmd_lstm': [124.00247955322266,
   126.00253295898438,
   125.00251007080078,
   125.50272369384766,
   124.00498962402344]},
 'FGEN': {'dmd_lstm': [18.382694244384766,
   18.10270118713379,
   18.182720184326172,
   18.022674560546875,
   18.385387420654297]},
 'ICT': {'dmd_lstm': [206.6023406982422,
   212.00233459472656,
   211.40199279785156,
   214.00294494628906,
   206.60494995117188]},
 'ALI': {'dmd_lstm': [29.052684783935547,
   29.202686309814453,
   29.102670669555664,
   29.2526912689209,
   29.05536651611328]},
 'SMC': {'dmd_lstm': [114.60272216796875,
   112.70266723632812,
   110.00323486328125,
   107.5022964477539,
   114.60485076904297]},
 'TEL': {'dmd_lstm': [1410.0040283203125,
   1395.002563476

In [11]:
def post_processing(processed_data, model_name='dmd_lstm', minimum_score=0.01):
    try:
        stock_symbols = stock_symbols = [
    'MEG',
    'JGS',
    'BDO',
    'FGEN',
    'ICT',
    'ALI',
    'SMC',
    'TEL',
    'GLO',
    'BLOOM',
    'RLC',
    'MER',
    'AC',
    'PGOLD',
    'LTG',
    'MPI',
    'AP',
    'RRHI',
    'URC'
]
        # Appeend PSEI to the list of stock symbols
        stock_symbols.append("PSEI")

        # Dictionary containing the last acutaul closing price of each stock
        last_actual_closing = {stock:processed_data[stock][0] for stock in stock_symbols}
        
        # Initialize the dictionaries for the stocks to buy and stocks to sell
        stocks_to_buy = {}
        stocks_to_sell = {}

        # Iterate through the processed data
        for stock in stock_symbols:
            predictions = processed_data[stock][1][model_name]
            # Calculate the percentile increase form last actual closing to the last prediction
            #   and add it to the stocks_to_buy or stocks_to_sell dictionary
            #   depending on the value of the percentile increase
            percentile_increase = (predictions[-1] - last_actual_closing[stock]) / last_actual_closing[stock]
            if percentile_increase > minimum_score:
                stocks_to_buy[stock] = processed_data[stock]
            else:
                stocks_to_sell[stock] = processed_data[stock]
        
        return stocks_to_buy, stocks_to_sell
    except Exception as e:
        print(e)

In [12]:
buy, sell = post_processing(processed_data)

In [13]:
buy.keys()

dict_keys(['FGEN', 'SMC', 'TEL', 'GLO', 'AC', 'URC'])

In [14]:
sell.keys()

dict_keys(['MEG', 'JGS', 'BDO', 'ICT', 'ALI', 'BLOOM', 'RLC', 'MER', 'PGOLD', 'LTG', 'MPI', 'AP', 'RRHI', 'PSEI'])

In [15]:
# Create the new files
from os import makedirs
from json import dump
makedirs("./json_data", exist_ok=True)

try:
    with open("./json_data/stocks2buy.json", "w") as f:
        dump(buy, f)
except:
    pass

try:
    with open("./json_data/stocks2sell.json", "w") as f:
        dump(sell, f)
    pass
except:
    pass

In [16]:
import json

In [31]:
stocks2buy = json.load(open("./json_data/stocks2buy.json"))
stocks2sell = json.load(open("./json_data/stocks2sell.json"))

In [18]:
from mongoengine import Document, StringField, FloatField, ListField, IntField, DictField
# Models for the database in MongoDB

# Define class "Buy" and inherit from "Document"
"""
About the Buy class:
    - This class is used to define the structure of the "buy" collection in the MongoDB database
    - This document contains the stocks to buy based from the predicted price trend from the Machine Learning model
"""
class Buy(Document):
    # Stock Symbol refers to the stock symbol of the stock
    stock_symbol =  StringField()
    # Last Closing refers to the last closing price of the stock
    last_closing =  FloatField()
    # Predicted Closing is a dictionary of predicted closing prices for the stock with a specific ML model used
    predicted_closing = DictField()

    # Format the data to JSON
    def to_json(self):
        return {
            "stock_symbol": self.stock_symbol,
            "last_closing": self.last_closing,
            "predicted_closing": self.predicted_closing
        }


# Define class "Sell" and inherit from "Document"
""""
About the Sell class:
    - This class is used to define the structure of the "sell" collection in the MongoDB database
    - This document contains the stocks to sell based from the predicted price trend from the Machine Learning model
"""
class Sell(Document):
    # Stock Symbol refers to the stock symbol of the stock
    stock_symbol =  StringField()
    # Last Closing refers to the last closing price of the stock
    last_closing =  FloatField()
    # Predicted Closing is a dictionary of predicted closing prices for the stock with a specific ML model used
    predicted_closing = DictField()

    # Format the data to JSON
    def to_json(self):
        return {
            "stock_symbol": self.stock_symbol,
            "last_closing": self.last_closing,
            "predicted_closing": self.predicted_closing
        }


# Define class "Info" and inherit from "Document"
"""
About the Info class:
    - This class is used to define the structure of the "info" collection in the MongoDB database
    - This document contains the general information of the stocks included in the alamSYS
"""
class Info(Document):
    # Stock Symbol refers to the stock symbol of the stock
    stock_symbol =  StringField()
    # Stock Name refers to the name of the company
    stock_name = StringField()
    # Comapany Site refers to the company's website
    company_site = StringField()
    # Company Address refers to the company's address
    company_address = StringField()
    # Company E-mail refers to the company's e-mail address
    company_email = StringField()
    # Company Phone refers to the company's phone number
    company_phone = IntField()
    # Sector refers to the sector of the company
    sector = StringField()
    # Industry refers to the industry of the company
    industry = StringField()
    # Key Executives is a  list of tuples of the key executives of the company and their positions
    key_executives = ListField()
    
    def to_json(self):
        return {
            "stock_symbol": self.stock_symbol,
            "stock_name": self.stock_name,
            "company_site": self.company_site,
            "company_address": self.company_address,
            "company_email": self.company_email,
            "company_phone": self.company_phone,
            "sector": self.sector,
            "industry": self.industry,
            "key_executives": self.key_executives
        }

# Define "ML_Models_Info" and inherit from "Document"
"""
About the ML_Models_Info class:
    - This class is used to define the structure of the "ml_models_info" collection in the MongoDB database
    - This document contains the information of the Machine Learning models used in the alamSYS
"""
class ML_Models_Info(Document):
    model_name = StringField()
    model_description = StringField()
    model_scores = DictField()

    def to_json(self):
        return {
            "model_name": self.model_name,
            "model_description": self.model_description,
            "model_scores": self.model_scores
        }

In [19]:
from mongoengine import connect
connect(db="alamAPI_DB", host="10.74.1.153", port=27017)

MongoClient(host=['10.74.1.153:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=3)

In [32]:
for stock in stocks2buy:
   Buy()

{'dmd_lstm': [2.1527161598205566, 2.1227190494537354, 2.0627264976501465, 2.042710542678833, 2.155424118041992]}
{'dmd_lstm': [54.502586364746094, 54.00267028808594, 52.20304870605469, 50.50254440307617, 54.50494384765625]}
{'dmd_lstm': [18.002695083618164, 17.782716751098633, 17.302783966064453, 17.062658309936523, 18.005321502685547]}
{'dmd_lstm': [206.00277709960938, 207.80172729492188, 209.80313110351562, 200.00283813476562, 206.0049285888672]}
{'dmd_lstm': [28.95272445678711, 28.202756881713867, 28.05269432067871, 28.50259780883789, 28.955408096313477]}
{'dmd_lstm': [1387.0040283203125, 1335.0003662109375, 1341.00927734375, 1280.996826171875, 1387.0018310546875]}
{'dmd_lstm': [1990.0050048828125, 1939.0013427734375, 1920.01025390625, 1870.9949951171875, 1990.0]}
{'dmd_lstm': [9.60273265838623, 9.292715072631836, 9.052791595458984, 8.6726713180542, 9.605388641357422]}
{'dmd_lstm': [15.502701759338379, 15.402709007263184, 15.202739715576172, 15.082682609558105, 15.505372047424316]}


In [33]:
for stock in stocks2sell:
    Sell(stock_symbol=stock, last_closing=stocks2sell[stock][0], predicted_closing=stocks2sell[stock][1]).save()